In [21]:
import boto3
import time
import io
import csv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import smtplib

Table_Name = 'DHT'

# create a DynamoDB resource
db = boto3.resource('dynamodb')
table = db.Table(Table_Name)
# Create a session with SageMaker
session = boto3.Session()
sagemaker = session.client('sagemaker-runtime')
try:
    while True:
        # get all items in the table
        response = table.scan()
        items = response['Items']
        
        # get the latest item from the table
        latest_item = max(items, key=lambda x: x['Sensor_Id'])
        print(latest_item)
        
        # Define the endpoint and input data
        endpoint_name = 'Your end point name'
        # lcohol	CH4	CO	H2	LPG	Propane	Smoke	Temp
        # 'SMOKE': '0.042945679550224644', 'Temperature': '34.0', 'H2': '0.010809321319730723', 'PROPANE': '0.014034855933218316', 'ALCHOHOL': '0.00595729435299386', 'Humidity': '58.0', 'CO': '0.010569839665393217', 'CH4': '0.0073075479519055485'
        data = [[latest_item['ALCHOHOL'], latest_item['CH4'], latest_item['CO'], latest_item['H2'], latest_item['LPG'], latest_item['PROPANE'], latest_item['SMOKE'], latest_item['Temperature']]]

        # Convert the input data to CSV format
        csv_buffer = io.StringIO()
        writer = csv.writer(csv_buffer)
        for row in data:
            writer.writerow(row)
        payload = csv_buffer.getvalue()

        # Send the request to the endpoint and get the response
        response = sagemaker.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=payload)

        # Parse the response and extract the predicted output
        result = response['Body'].read().decode('utf-8')
        predicted_output = float(result.strip())
        print(predicted_output)
        # check temperature for emergency alert
        if predicted_output > 0.5:
            # email parameters
            sender_email = 'email'
            sender_password = 'email app password'
            recipient_email = 'email'
            subject = 'Emergency Alert!'
            body = 'There is a possibility gas leakage in your house.'

            # create email message
            message = MIMEMultipart()
            message['From'] = sender_email
            message['To'] = recipient_email
            message['Subject'] = subject

            # add body to email
            message.attach(MIMEText(body, 'plain'))

            # send email
            try:
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.ehlo()
                server.starttls()
                server.login(sender_email, sender_password)
                server.sendmail(sender_email, recipient_email, message.as_string())
                server.close()

                print('Email sent successfully')

            except Exception as e:
                print('Something went wrong:', e)

        time.sleep(5)

except KeyboardInterrupt:
    print()
    print('Aborted by user!')


{'Sensor_Id': Decimal('299'), 'LPG': '0.013477921192778129', 'SMOKE': '0.042945679550224644', 'Temperature': '34.0', 'H2': '0.010809321319730723', 'PROPANE': '0.014034855933218316', 'ALCHOHOL': '0.00595729435299386', 'Humidity': '58.0', 'CO': '0.010569839665393217', 'CH4': '0.0073075479519055485'}
0.021793659776449203
{'Sensor_Id': Decimal('299'), 'LPG': '0.013477921192778129', 'SMOKE': '0.042945679550224644', 'Temperature': '34.0', 'H2': '0.010809321319730723', 'PROPANE': '0.014034855933218316', 'ALCHOHOL': '0.00595729435299386', 'Humidity': '58.0', 'CO': '0.010569839665393217', 'CH4': '0.0073075479519055485'}
0.021793659776449203
{'Sensor_Id': Decimal('299'), 'LPG': '0.013477921192778129', 'SMOKE': '0.042945679550224644', 'Temperature': '34.0', 'H2': '0.010809321319730723', 'PROPANE': '0.014034855933218316', 'ALCHOHOL': '0.00595729435299386', 'Humidity': '58.0', 'CO': '0.010569839665393217', 'CH4': '0.0073075479519055485'}
0.021793659776449203

Aborted by user!
